In [1]:
import sys
sys.path.insert(0, '/home/jonathan/Documents/mi3-gs/build/python')

import drjit as dr
from drjit.auto.ad import Float, UInt
import mitsuba as mi
mi.set_variant('cuda_ad_rgb')

jit_find_library(): Unable to load "/usr/lib/llvm-15/lib/libLLVM.so": /home/jonathan/miniconda3/envs/volprim/lib/python3.11/site-packages/zmq/backend/cython/../../../../.././libstdc++.so.6: version `GLIBCXX_3.4.30' not found (required by /usr/lib/llvm-15/lib/libLLVM.so)!


### Generate scene renders
Generates synthetic renders and camera poses to be used as input for a NeRF/GS surface reconstruction algorithm.

In [2]:
from data_generation import CameraParameters, DataGenerator, make_hemispherical_cameras, make_spherical_cameras, concatenate_cameras
import numpy as np

xs = np.linspace(-3, 3, 5)
zs = np.linspace(-4.5, 4.5, 5)
xx, zz = np.meshgrid(xs, zs)
centers = np.c_[xx.ravel(), zz.ravel()]

scene = mi.load_file("../../scenes/mitsuba/classroom/scene.xml")
camera_params = CameraParameters(512, 512, fov=90, spp=64)
camera_poses = concatenate_cameras([
    make_spherical_cameras(center = np.array([x, 1.4, z]), radius = 0.2, outward = True, density = 0) for \
        x, z in zip(xx.ravel(), zz.ravel())])

datagen = DataGenerator(scene, "classroom_spp=64_dn", camera_params, camera_poses, use_denoiser=True)
# datagen.visualize()
# datagen.run()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
scene = mi.load_dict(mi.cornell_box())
camera_params = CameraParameters(512, 512, 64, 90)
camera_poses = make_spherical_cameras(center = np.array([0.0, 0.25, 0.75]), radius = 0.5, outward = False, density = 2)

datagen = DataGenerator(scene, "cbox", camera_params, camera_poses, use_denoiser=False)
# datagen.visualize()
datagen.run()

Starting render job for training data ...


100%|██████████| 320/320 [00:11<00:00, 27.25it/s]


Training data render complete.
Starting render job for test data ...


100%|██████████| 80/80 [00:03<00:00, 25.78it/s]


Test data render complete.


In [5]:
scene = mi.load_file("../../scenes/mitsuba/lego/scene.xml")
camera_params = CameraParameters(512, 512, 64, 45)
camera_poses = make_hemispherical_cameras(center = np.array([0.5, 0.25, 0.5]), radius = 1.0, outward = False, tophalf = True, density = 2)

datagen = DataGenerator(scene, "lego", camera_params, camera_poses, use_denoiser=False)
# datagen.visualize()
datagen.run()

Starting render job for training data ...


100%|██████████| 158/158 [00:29<00:00,  5.40it/s]


Training data render complete.
Starting render job for test data ...


100%|██████████| 40/40 [00:07<00:00,  5.45it/s]


Test data render complete.


### 3DGS visualization
Overlay the trained 3DGS splats on top of the original Mitsuba scene.

In [22]:
from plyfile import PlyData
import numpy as np

scene_fp = "../../scenes/mitsuba/classroom/scene.xml"
scene = mi.load_file(scene_fp)
data_fp = "/home/jonathan/Documents/gaussian-splatting/output/mitsuba-classroom-64spp/point_cloud/iteration_30000/point_cloud.ply"

# scene = mi.load_dict(mi.cornell_box())
# data_fp = "/home/jonathan/Documents/gaussian-splatting/output/mitsuba-cbox/point_cloud/iteration_30000/point_cloud.ply"

with open(data_fp, 'rb') as f:
    plydata = PlyData.read(f)['vertex']
    points = np.c_[plydata['x'], plydata['y'], plydata['z']]
    colors = np.c_[plydata['f_dc_0'], plydata['f_dc_1'], plydata['f_dc_2']]

In [28]:
import polyscope as ps

ps.init()
ps_points = ps.register_point_cloud("GS points", points)
ps_points.add_color_quantity("Colors", colors, enabled=True)

for shape_id, shape in enumerate(scene.shapes()):
    if not(shape.is_mesh()): continue

    V = dr.unravel(mi.Point3f, shape.vertex_positions_buffer())
    F = dr.unravel(mi.Point3u, shape.faces_buffer())
    V, F = V.numpy().T, F.numpy().T
    ps.register_surface_mesh(f"mesh_{shape_id}", V, F, edge_width=1.0)

ps.show()

ps.remove_point_cloud("GS points")
for shape_id, shape in enumerate(scene.shapes()):
    try:
        ps.remove_surface_mesh(f"mesh_{shape_id}")
    except:
        pass